In [2]:
import pandas as pd
import numpy as np
import re
import keras
from keras.models import Sequential
from keras.layers import Dense


Using TensorFlow backend.


In [62]:
import tensorflow as tf
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
sess

In [2]:
# dropping @username, changing sentiment to boolean, dropping empty tweets
df = pd.read_csv('training.1600000.processed.noemoticon.csv',encoding = "ISO-8859-1", header = None)
pattern = re.compile('@[A-Za-z0-9_]*')
df[6] = df[5].replace(pattern,"")
df[0] = df[0] == 4
df.drop([1,2,3,4],axis = 1, inplace = True)
df.drop(df[df[6].str.isspace()].index, inplace = True)
df.columns = ['sentiment','original','final']

In [45]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [3]:
# remove punctuation and capital letters
import string
df['no_punc'] = df['final'].str.replace('[{}]'.format(string.punctuation), '').str.lower()
df['split'] = df['no_punc'].str.split()
df.head(10)

,sentiment,original,final,no_punc,split
0,False,"@switchfoot http://twitpic.com/2y1zl - Awww, t...","http://twitpic.com/2y1zl - Awww, that's a bum...",httptwitpiccom2y1zl awww thats a bummer you...,"[httptwitpiccom2y1zl, awww, thats, a, bummer, ..."
1,False,is upset that he can't update his Facebook by ...,is upset that he can't update his Facebook by ...,is upset that he cant update his facebook by t...,"[is, upset, that, he, cant, update, his, faceb..."
2,False,@Kenichan I dived many times for the ball. Man...,I dived many times for the ball. Managed to s...,i dived many times for the ball managed to sa...,"[i, dived, many, times, for, the, ball, manage..."
3,False,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire,"[my, whole, body, feels, itchy, and, like, its..."
4,False,"@nationwideclass no, it's not behaving at all....","no, it's not behaving at all. i'm mad. why am...",no its not behaving at all im mad why am i he...,"[no, its, not, behaving, at, all, im, mad, why..."
5,False,@Kwesidei not the whole crew,not the whole crew,not the whole crew,"[not, the, whole, crew]"
6,False,Need a hug,Need a hug,need a hug,"[need, a, hug]"
7,False,@LOLTrish hey long time no see! Yes.. Rains a...,"hey long time no see! Yes.. Rains a bit ,onl...",hey long time no see yes rains a bit only a ...,"[hey, long, time, no, see, yes, rains, a, bit,..."
8,False,@Tatiana_K nope they didn't have it,nope they didn't have it,nope they didnt have it,"[nope, they, didnt, have, it]"
9,False,@twittera que me muera ?,que me muera ?,que me muera,"[que, me, muera]"



### Positive and Negative Word Counts

In [32]:
# positive and negative word lists
with open("positive-words.txt") as f:
    content = f.readlines()
content
positive_words = [line.strip() for line in content if line[0] != ';' and not line.isspace()]
positive_words[0:5]

['a+', 'abound', 'abounds', 'abundance', 'abundant']

In [33]:
with open("negative-words.txt") as f:
    content = f.readlines()
content
negative_words = [line.strip() for line in content if line[0] != ';' and not line.isspace()]
negative_words[0:5]

['2-faced', '2-faces', 'abnormal', 'abolish', 'abominable']

In [89]:
from collections import Counter

def categorize_string(tweet):
    positive_count = 0
    negative_count = 0
    count = Counter(tweet.split())
    for key,val in count.items():
        if key in positive_words:
            positive_count += 1
        if key in negative_words:
            negative_count += 1
    if positive_count > negative_count:
        return 1
    if negative_count > positive_count:
        return 0
    else:
        return -5

In [90]:
df["count_metric"] = df['no_punc'].apply(categorize_string)
df

,sentiment,original,final,no_punc,split,count_metric
0,False,"@switchfoot http://twitpic.com/2y1zl - Awww, t...","http://twitpic.com/2y1zl - Awww, that's a bum...",httptwitpiccom2y1zl awww thats a bummer you...,"[httptwitpiccom2y1zl, awww, thats, a, bummer, ...",-5
1,False,is upset that he can't update his Facebook by ...,is upset that he can't update his Facebook by ...,is upset that he cant update his facebook by t...,"[is, upset, that, he, cant, update, his, faceb...",0
2,False,@Kenichan I dived many times for the ball. Man...,I dived many times for the ball. Managed to s...,i dived many times for the ball managed to sa...,"[i, dived, many, times, for, the, ball, manage...",-5
3,False,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire,"[my, whole, body, feels, itchy, and, like, its...",-5
4,False,"@nationwideclass no, it's not behaving at all....","no, it's not behaving at all. i'm mad. why am...",no its not behaving at all im mad why am i he...,"[no, its, not, behaving, at, all, im, mad, why...",0
5,False,@Kwesidei not the whole crew,not the whole crew,not the whole crew,"[not, the, whole, crew]",-5
6,False,Need a hug,Need a hug,need a hug,"[need, a, hug]",1
7,False,@LOLTrish hey long time no see! Yes.. Rains a...,"hey long time no see! Yes.. Rains a bit ,onl...",hey long time no see yes rains a bit only a ...,"[hey, long, time, no, see, yes, rains, a, bit,...",1
8,False,@Tatiana_K nope they didn't have it,nope they didn't have it,nope they didnt have it,"[nope, they, didnt, have, it]",-5
9,False,@twittera que me muera ?,que me muera ?,que me muera,"[que, me, muera]",-5


In [83]:
df[df['count_metric']== 0]['count_metric'] = -5

In [84]:
df[df['count_metric']==-1]['metric'] = 0

In [91]:
len(df[df['count_metric']==df['sentiment']])/len(df)

0.4115612719364971

### Naive Bayes and TF-IDF

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer

In [25]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['no_punc'])

In [26]:
y = df['sentiment']
X = X.tocsc()

In [94]:
X_traincv, X_test, y_traincv, y_test = train_test_split(X,y, test_size = .2, random_state = 3)
X_train, X_cv, y_train, y_cv = train_test_split(X_traincv, y_traincv, test_size = .25, random_state = 3)
multi = MultinomialNB()
model = multi.fit(X_train,y_train)

In [95]:
print(model.score(X_test,y_test))
print(model.score(X_train,y_train))

0.780167596891
0.823243263548


#### Grid Search for min_df parameter

In [98]:
min_dfs = [.000001,0.000005, .00001]
for mdf in min_dfs:
    vectorizer1 = CountVectorizer(min_df = mdf)
    X = vectorizer1.fit_transform(df['no_punc'])
    y = df['sentiment']
    X_traincv, X_test, y_traincv, y_test = train_test_split(X,y, test_size = .2, random_state = 3)
    X_train, X_cv, y_train, y_cv = train_test_split(X_traincv, y_traincv, test_size = .25, random_state = 3)
    modeldf = multi.fit(X_train,y_train)
    print(mdf, modeldf.score(X_cv,y_cv))

1e-06 0.780483071221
5e-06 0.78084618852
1e-05 0.780254557748


In [26]:
best_min_df = .000005

#### Grid Search for alpha

In [7]:
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import accuracy_score

A:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
A:\Anaconda3\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [27]:
vectorizer1 = CountVectorizer(min_df = best_min_df)
X = vectorizer1.fit_transform(df['no_punc'])
y = df['sentiment']
X_traincv, X_test, y_traincv, y_test = train_test_split(X,y, test_size = .2, random_state = 3)
X_train, X_cv, y_train, y_cv = train_test_split(X_traincv, y_traincv, test_size = .25, random_state = 3)

model2 = MultinomialNB()
alphas = [.1,.5,1,5,10]
for a in alphas:
    model2 = MultinomialNB(alpha = a)
    model2.fit(X_train,y_train)
    print(a,accuracy_score(model2.predict(X_cv),y_cv))

0.1 0.779506410899
0.5 0.780210733247
1 0.78084618852
5 0.781528598618
10 0.78112791746


In [102]:
model3 = MultinomialNB(alpha = 5.0)
model3.fit(X_train,y_train)
accuracy_score(model3.predict(X_test),y_test)

0.78186109554650551

#### Using English Stop Words

In [30]:
vectorizer_stopwords = CountVectorizer(min_df = best_min_df, stop_words = 'english')
Xsw = vectorizer_stopwords.fit_transform(df['no_punc'])
ysw = df['sentiment']
Xsw_traincv, Xsw_test, ysw_traincv, ysw_test = train_test_split(Xsw,ysw, test_size = .2, random_state = 3)
Xsw_train, Xsw_cv, ysw_train, ysw_cv = train_test_split(Xsw_traincv, ysw_traincv, test_size = .25, random_state = 3)
model3 = MultinomialNB(alpha = 5.0).fit(X_train,y_train)
model_stopwords = MultinomialNB(alpha = 5.0).fit(Xsw_train,ysw_train)
print(accuracy_score(model3.predict(X_cv),y_cv),accuracy_score(model_stopwords.predict(Xsw_cv),ysw_cv))

0.781528598618 0.767113467895


In [36]:
min_dfs = [.000001,0.000005, .00001]
for mdf in min_dfs:
    vectorizer1 = CountVectorizer(min_df = mdf, stop_words = 'english')
    Xsw = vectorizer1.fit_transform(df['no_punc'])
    ysw = df['sentiment']
    Xsw_traincv, Xsw_test, ysw_traincv, ysw_test = train_test_split(Xsw,ysw, test_size = .2, random_state = 3)
    Xsw_train, Xsw_cv, ysw_train, ysw_cv = train_test_split(Xsw_traincv, ysw_traincv, test_size = .25, random_state = 3)
    modeldf = MultinomialNB().fit(Xsw_train,ysw_train)
    print(mdf, modeldf.score(Xsw_cv,ysw_cv))

1e-06 0.766061679856
5e-06 0.766262020435
1e-05 0.765895772814


In [33]:
alphas = [.1,.5,1,5,10]
for a in alphas:
    model_stopwords = MultinomialNB(alpha = a)
    model_stopwords.fit(Xsw_train,ysw_train)
    print(a,accuracy_score(model_stopwords.predict(Xsw_cv),ysw_cv))

0.1 0.764950415707
0.5 0.765920815386
1 0.766262020435
5 0.767113467895
10 0.767060252429


Using stop words result in a worse sentiment classifier.

#### TF-IDF

In [61]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfvectorizer = TfidfVectorizer(min_df=1)
Xtfidf=tfidfvectorizer.fit_transform(df['no_punc'])
ytfidf = df['sentiment']
Xtfidf_traincv, Xtfidf_test, ytfidf_traincv, ytfidf_test = train_test_split(Xtfidf,ytfidf, test_size = .2, random_state = 3)
Xtfidf_train, Xtfidf_cv, ytfidf_train, ytfidf_cv = train_test_split(Xtfidf_traincv, ytfidf_traincv, test_size = .25, random_state = 3)

In [38]:
model_tfidf = MultinomialNB().fit(Xtfidf_train,ytfidf_train)
accuracy_score(model_tfidf.predict(Xtfidf_test),ytfidf_test)

0.77247016030326465

In [40]:
min_dfs = [.000005,.00001,.00005,.0001]
for mdf in min_dfs:
    tfidfvectorizer1 = TfidfVectorizer(min_df = mdf)
    Xtfidf = tfidfvectorizer1.fit_transform(df['no_punc'])
    ytfidf = df['sentiment']
    Xtfidf_traincv, Xtfidf_test, ytfidf_traincv, ytfidf_test = train_test_split(Xtfidf,ytfidf, test_size = .2, random_state = 3)
    Xtfidf_train, Xtfidf_cv, ytfidf_train, ytfidf_cv = train_test_split(Xtfidf_traincv, ytfidf_traincv, test_size = .25, random_state = 3)
    modeldf = MultinomialNB().fit(Xtfidf_train,ytfidf_train)
    print(mdf, accuracy_score(modeldf.predict(Xtfidf_cv),ytfidf_cv))

5e-06 0.773756010217
1e-05 0.774103475909
5e-05 0.772594660924
0.0001 0.77076655314


In [16]:
best_min_df_tfidf = .00001

In [20]:
vectorizer1 = TfidfVectorizer(min_df = best_min_df_tfidf)
X = vectorizer1.fit_transform(df['no_punc'])
y = df['sentiment']
Xtfidf_traincv, Xtfidf_test, ytfidf_traincv, ytfidf_test = train_test_split(Xtfidf,ytfidf, test_size = .2, random_state = 3)
Xtfidf_train, Xtfidf_cv, ytfidf_train, ytfidf_cv = train_test_split(Xtfidf_traincv, ytfidf_traincv, test_size = .25, random_state = 3)

alphas = [.5,1.0,5.0,10.0,50.0,100.0,500.]
for a in alphas:
    model2 = MultinomialNB(alpha = a)
    model2.fit(Xtfidf_train,ytfidf_train)
    print(a,accuracy_score(model2.predict(Xtfidf_cv),ytfidf_cv))

0.5 0.755900656115
1.0 0.755960132225
5.0 0.756467244315
10.0 0.756914880297
50.0 0.758289091455
100.0 0.75783206451
500.0 0.751490033056


In [21]:
best_alpha_tfidf = 50.0

In [41]:
vectorizer1 = TfidfVectorizer(min_df = best_min_df_tfidf)
model3 = MultinomialNB(alpha = best_alpha_tfidf)
model3.fit(Xtfidf_train,ytfidf_train)
accuracy_score(model3.predict(Xtfidf_test),ytfidf_test)

0.77340612351584093

### LDA and NMF

In [4]:
from sklearn.decomposition import NMF, LatentDirichletAllocation

In [65]:
n_features = 1000
tf_vectorizer = CountVectorizer(max_features = n_features)
X_lda = tf_vectorizer.fit_transform(df['no_punc'])

In [66]:
n_topics = 100
lda = LatentDirichletAllocation(n_topics = n_topics, random_state = 3, learning_method = 'online').fit_transform(X_lda)

In [69]:
y = df['sentiment']
Xlda_traincv, Xlda_test, ylda_traincv, ylda_test = train_test_split(lda,y, test_size = .2, random_state = 3)
Xlda_train, Xlda_cv, ylda_train, ylda_cv = train_test_split(Xlda_traincv, ylda_traincv, test_size = .25, random_state = 3)

In [70]:
lda_nb = MultinomialNB().fit(Xlda_train, ylda_train)

In [71]:
accuracy_score(lda_nb.predict(Xlda_test),ylda_test)

0.63244505520304772

In [ ]:
tf_vectorizer = CountVectorizer()
X_lda = tf_vectorizer.fit_transform(df['no_punc'])
n_topics = 100
lda = LatentDirichletAllocation(n_topics = n_topics, random_state = 3, learning_method = 'online').fit_transform(X_lda)
Xlda_traincv, Xlda_test, ylda_traincv, ylda_test = train_test_split(lda,y, test_size = .2, random_state = 3)
Xlda_train, Xlda_cv, ylda_train, ylda_cv = train_test_split(Xlda_traincv, ylda_traincv, test_size = .25, random_state = 3)
lda_nb2 = MultinomialNB().fit(Xlda_train, ylda_train)
accuracy_score(lda_nb2.predict(Xlda_test),ylda_test)

In [72]:
nmf_vectorizer = TfidfVectorizer(max_features = n_features)
X_nmf = nmf_vectorizer.fit_transform(df['no_punc'])
nmf = NMF(n_components = n_topics, random_state = 3).fit_transform(X_nmf)

In [73]:
y = df['sentiment']
Xnmf_traincv, Xnmf_test, ynmf_traincv, ynmf_test = train_test_split(nmf,y, test_size = .2, random_state = 3)
Xnmf_train, Xnmf_cv, ynmf_train, ynmf_cv = train_test_split(Xnmf_traincv, ynmf_traincv, test_size = .25, random_state = 3)

In [74]:
nmf_nb = MultinomialNB().fit(Xnmf_train,ynmf_train)
accuracy_score(nmf_nb.predict(Xnmf_test),ynmf_test)

0.6809429751108913

In [ ]:
nmf_vectorizer = TfidfVectorizer()
X_nmf = nmf_vectorizer.fit_transform(df['no_punc'])
nmf = NMF(n_components = n_topics, random_state = 3).fit_transform(X_nmf)
Xnmf_traincv, Xnmf_test, ynmf_traincv, ynmf_test = train_test_split(nmf,y, test_size = .2, random_state = 3)
Xnmf_train, Xnmf_cv, ynmf_train, ynmf_cv = train_test_split(Xnmf_traincv, ynmf_traincv, test_size = .25, random_state = 3)
nmf_nb = MultinomialNB().fit(Xnmf_train,ynmf_train)
accuracy_score(nmf_nb.predict(Xnmf_test),ynmf_test)

### Feed Forward Neural Network

#### No Embedding Layer, 25 node GloVe vectors, Vector Sum Method

In [4]:
vector_len = 25
tokens = df['split']

#Load the Glove model into Python
def loadGloveModel(gloveFile):
    print("Loading GloVe Model")
    with open(gloveFile,'r',encoding="utf8") as f:
        model = {}
        for line in f:
            splitLine = line.split()
            word = splitLine[0]
            embedding = [float(val) for val in splitLine[1:]]
            model[word] = embedding
        print("Done.",len(model)," words loaded!")
    return model
 
gloVe = loadGloveModel('glove.twitter.27B.25d.txt')
 
def word_embed(tokens):
    embeddings = []
    for i in range(len(tokens)):
        try:
            embeddings.append(gloVe[tokens[i]])
        except:
            pass
    return(embeddings)
word_embeddings = [word_embed(t) for t in tokens]
 
 
def doc_vectorizer(word_embeddings, vector_len):
    sent_vec = np.zeros(vector_len)
    for w in word_embeddings:
        try:
            sent_vec = np.add(sent_vec, w)
        except:
            pass
    return sent_vec
doc_embeddings = [doc_vectorizer(w,vector_len) for w in word_embeddings]



Loading GloVe Model
Done. 1193514  words loaded!


In [9]:
from keras.utils.np_utils import to_categorical
X = np.array(doc_embeddings)
y = to_categorical(np.asarray(df['sentiment'].values.tolist()))
X_traincv, X_test, y_traincv, y_test = train_test_split(X,y, test_size = .2, random_state = 3)
X_train, X_cv, y_train, y_cv = train_test_split(X_traincv, y_traincv, test_size = .25, random_state = 3)

In [11]:
from keras.callbacks import EarlyStopping

model_nn = Sequential()
model_nn.add(Dense(25,activation = 'relu', input_shape = (X_train.shape[1],)))
model_nn.add(Dense(2, activation = 'sigmoid'))
early_stopping = EarlyStopping(patience = 2)
model_nn.compile(optimizer = 'adam',loss = 'binary_crossentropy', metrics = ['accuracy'],callbacks = [early_stopping])

In [12]:
model_nn.fit(X_train,y_train)

Epoch 1/10
958368/958368 [==============================] - 39s - loss: 0.5917 - acc: 0.6835    - ETA: 1s - loss: 0.
Epoch 2/10
958368/958368 [==============================] - 37s - loss: 0.5762 - acc: 0.6942    - ETA:
Epoch 3/10
958368/958368 [==============================] - 33s - loss: 0.5727 - acc: 0.6970    
Epoch 4/10
958368/958368 [==============================] - 32s - loss: 0.5710 - acc: 0.6989    - ETA: 0s - loss: 0.5711 - acc
Epoch 5/10
958368/958368 [==============================] - 29s - loss: 0.5699 - acc: 0.6997    - E - ETA: 0s - loss: 0.5699 - acc: 0.69 - ETA: 0s - loss: 0.5699 - acc: 0.699
Epoch 6/10
958368/958368 [==============================] - 28s - loss: 0.5691 - acc: 0.7002    - ETA: 4s -  - ETA
Epoch 7/10
958368/958368 [==============================] - 30s - loss: 0.5686 - acc: 0.7011    - ETA: 9s -  - ETA: 4s - loss: 0.5685 - acc: 0.70 - ETA: - ETA: 0s - loss: 0.5686 - ac
Epoch 8/10
958368/958368 [==============================] - 28s - loss: 0.5681 - ac

In [13]:
model_nn.evaluate(X_test,y_test)

319457/319457 [==============================] - 5s     


[0.56880097532597584, 0.70029769264721076]

#### Embedding Layer, 25 node GloVe Vector

In [14]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
 
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['no_punc'].values.tolist())
sequences = tokenizer.texts_to_sequences(df['no_punc'].values.tolist())
 
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
 


Found 516594 unique tokens.


In [15]:
from keras.utils.np_utils import to_categorical
 
data = pad_sequences(sequences)
labels = to_categorical(np.asarray(df['sentiment'].values.tolist()))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)
input_len = data.shape[1]

Shape of data tensor: (1597281, 40)
Shape of label tensor: (1597281, 2)


In [16]:
# Split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(0.2 * data.shape[0])
 
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]
 


In [17]:
vector_len = 25
embeddings_index = {}
with open('glove.twitter.27B.25d.txt', 'r',encoding='utf8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
print('Found %s word vectors.' % len(embeddings_index))
 
embedding_matrix = np.zeros((len(word_index) + 1, vector_len))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
 


Found 1193514 word vectors.


In [25]:
embedding_matrix.shape

(516595, 25)

In [26]:
from keras.layers import Embedding
embedding_layer = Embedding(len(word_index) + 1,
                            vector_len,
                            weights=[embedding_matrix],
                            input_length=input_len,
                            #batch_size=3,
                            trainable=False)
 


In [33]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.layers import Flatten

early_stopping = EarlyStopping(patience = 2)
model = Sequential()
model.add(embedding_layer)
model.add(Flatten())
model.add(Dense(25,activation='relu'))
model.add(Dense(2, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=10,callbacks = [early_stopping])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 40, 25)            12914875  
_________________________________________________________________
flatten_2 (Flatten)          (None, 1000)              0         
_________________________________________________________________
dense_15 (Dense)             (None, 25)                25025     
_________________________________________________________________
dense_16 (Dense)             (None, 2)                 52        
Total params: 12,939,952
Trainable params: 25,077
Non-trainable params: 12,914,875
_________________________________________________________________
None
Train on 1277825 samples, validate on 319456 samples
Epoch 1/10
1277825/1277825 [==============================] - 64s - loss: 0.5683 - acc: 0.6988 - val_loss: 0.5552 - val_acc: 0.7095
Epoch 2/10
1277825/1277825 [==============================] - 64s -

In [35]:
model.evaluate(x_val,y_val)

317888/319456 [============================>.] - ETA: 0s

[0.54206403729004027, 0.72171128418311126]

#### Embedding Layer, 50 node GloVe Vector

In [14]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['no_punc'].values.tolist())
sequences = tokenizer.texts_to_sequences(df['no_punc'].values.tolist())
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
data = pad_sequences(sequences)
labels = to_categorical(np.asarray(df['sentiment'].values.tolist()))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)
input_len = data.shape[1]

Found 516594 unique tokens.


In [36]:
# Split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(0.2 * data.shape[0])
 
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]
 


In [37]:
vector_len = 50
embeddings_index = {}
with open('glove.twitter.27B.50d.txt', 'r',encoding='utf8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
print('Found %s word vectors.' % len(embeddings_index))
 
embedding_matrix = np.zeros((len(word_index) + 1, vector_len))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

Found 1193514 word vectors.


In [38]:
embedding_layer = Embedding(len(word_index) + 1,
                            vector_len,
                            weights=[embedding_matrix],
                            input_length=input_len,
                            #batch_size=3,
                            trainable=False)
early_stopping = EarlyStopping(patience = 2)

model = Sequential()
model.add(embedding_layer)
model.add(Flatten())
model.add(Dense(25,activation='relu'))
model.add(Dense(2, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=10,callbacks = [early_stopping])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 40, 50)            25829750  
_________________________________________________________________
flatten_3 (Flatten)          (None, 2000)              0         
_________________________________________________________________
dense_17 (Dense)             (None, 25)                50025     
_________________________________________________________________
dense_18 (Dense)             (None, 2)                 52        
Total params: 25,879,827
Trainable params: 50,077
Non-trainable params: 25,829,750
_________________________________________________________________
None
Train on 1277825 samples, validate on 319456 samples
Epoch 1/10
1277825/1277825 [==============================] - 83s - loss: 0.5230 - acc: 0.7351 - val_loss: 0.5096 - val_acc: 0.7449
Epoch 2/10
1277825/1277825 [==============================] - 82s -

#### Embedding Layer, 100 node GloVe Vector

In [39]:
vector_len = 100
embeddings_index = {}
with open('glove.twitter.27B.100d.txt', 'r',encoding='utf8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
print('Found %s word vectors.' % len(embeddings_index))
 
embedding_matrix = np.zeros((len(word_index) + 1, vector_len))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

Found 1193514 word vectors.


In [40]:
embedding_layer = Embedding(len(word_index) + 1,
                            vector_len,
                            weights=[embedding_matrix],
                            input_length=input_len,
                            #batch_size=3,
                            trainable=False)
early_stopping = EarlyStopping(patience = 2)

model = Sequential()
model.add(embedding_layer)
model.add(Flatten())
model.add(Dense(25,activation='relu'))
model.add(Dense(2, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=10,callbacks = [early_stopping])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 40, 100)           51659500  
_________________________________________________________________
flatten_4 (Flatten)          (None, 4000)              0         
_________________________________________________________________
dense_19 (Dense)             (None, 25)                100025    
_________________________________________________________________
dense_20 (Dense)             (None, 2)                 52        
Total params: 51,759,577
Trainable params: 100,077
Non-trainable params: 51,659,500
_________________________________________________________________
None
Train on 1277825 samples, validate on 319456 samples
Epoch 1/10
1277825/1277825 [==============================] - 103s - loss: 0.4941 - acc: 0.7571 - val_loss: 0.4794 - val_acc: 0.7670
Epoch 2/10
1277825/1277825 [==============================] - 102

#### Embedding Layer, 200 node GloVe Vector

In [41]:
vector_len = 200
embeddings_index = {}
with open('glove.twitter.27B.200d.txt', 'r',encoding='utf8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
print('Found %s word vectors.' % len(embeddings_index))
 
embedding_matrix = np.zeros((len(word_index) + 1, vector_len))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

Found 1193514 word vectors.


In [42]:
embedding_layer = Embedding(len(word_index) + 1,
                            vector_len,
                            weights=[embedding_matrix],
                            input_length=input_len,
                            #batch_size=3,
                            trainable=False)
early_stopping = EarlyStopping(patience = 2)

model = Sequential()
model.add(embedding_layer)
model.add(Flatten())
model.add(Dense(25,activation='relu'))
model.add(Dense(2, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=10,callbacks = [early_stopping])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 40, 200)           103319000 
_________________________________________________________________
flatten_5 (Flatten)          (None, 8000)              0         
_________________________________________________________________
dense_21 (Dense)             (None, 25)                200025    
_________________________________________________________________
dense_22 (Dense)             (None, 2)                 52        
Total params: 103,519,077
Trainable params: 200,077
Non-trainable params: 103,319,000
_________________________________________________________________
None
Train on 1277825 samples, validate on 319456 samples
Epoch 1/10
1277825/1277825 [==============================] - 154s - loss: 0.4757 - acc: 0.7695 - val_loss: 0.4618 - val_acc: 0.7788
Epoch 2/10
1277825/1277825 [==============================] - 1

### Recurrent Neural Network

#### 25 node GloVe layer

In [55]:
vector_len = 25
embeddings_index = {}
with open('glove.twitter.27B.25d.txt', 'r',encoding='utf8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
print('Found %s word vectors.' % len(embeddings_index))
 
embedding_matrix = np.zeros((len(word_index) + 1, vector_len))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

Found 1193514 word vectors.


In [60]:
from keras.layers import LSTM
early_stopping = EarlyStopping(patience = 2)
embedding_layer = Embedding(len(word_index) + 1,
                            vector_len,
                            weights=[embedding_matrix],
                            input_length=input_len,
                            #batch_size=3,
                            trainable=False)

model_rnn = Sequential()
model_rnn.add(embedding_layer)
model_rnn.add(LSTM(100))
model_rnn.add(Dense(25,activation='relu'))
model_rnn.add(Dense(2, activation='sigmoid'))
model_rnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model_rnn.summary())
model_rnn.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=10, batch_size = 256, callbacks = [early_stopping])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, 40, 25)            12914875  
_________________________________________________________________
lstm_11 (LSTM)               (None, 100)               50400     
_________________________________________________________________
dense_42 (Dense)             (None, 25)                2525      
_________________________________________________________________
dense_43 (Dense)             (None, 2)                 52        
Total params: 12,967,852
Trainable params: 52,977
Non-trainable params: 12,914,875
_________________________________________________________________
None
Train on 1277825 samples, validate on 319456 samples
Epoch 1/10
1277825/1277825 [==============================] - 1080s - loss: 0.4988 - acc: 0.7527 - val_loss: 0.4605 - val_acc: 0.7804
Epoch 2/10
1277825/1277825 [==============================] - 106

#### 50 node GLoVe layer

In [45]:
vector_len = 50
embeddings_index = {}
with open('glove.twitter.27B.50d.txt', 'r',encoding='utf8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
print('Found %s word vectors.' % len(embeddings_index))
 
embedding_matrix = np.zeros((len(word_index) + 1, vector_len))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

Found 1193514 word vectors.


In [46]:
from keras.layers import LSTM
early_stopping = EarlyStopping(patience = 2)
embedding_layer = Embedding(len(word_index) + 1,
                            vector_len,
                            weights=[embedding_matrix],
                            input_length=input_len,
                            #batch_size=3,
                            trainable=False)

model_rnn = Sequential()
model_rnn.add(embedding_layer)
model_rnn.add(LSTM(100))
model_rnn.add(Dense(25,activation='relu'))
model_rnn.add(Dense(2, activation='sigmoid'))
model_rnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model_rnn.summary())
model_rnn.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=10, batch_size = 256, callbacks = [early_stopping])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 40, 50)            25829750  
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               60400     
_________________________________________________________________
dense_25 (Dense)             (None, 25)                2525      
_________________________________________________________________
dense_26 (Dense)             (None, 2)                 52        
Total params: 25,892,727
Trainable params: 62,977
Non-trainable params: 25,829,750
_________________________________________________________________
None
Train on 1277825 samples, validate on 319456 samples
Epoch 1/10
1277825/1277825 [==============================] - 979s - loss: 0.4549 - acc: 0.7831 - val_loss: 0.4216 - val_acc: 0.8050
Epoch 2/10
1277825/1277825 [==============================] - 909s

#### 100 node GLoVe layer

In [47]:
vector_len = 100
embeddings_index = {}
with open('glove.twitter.27B.100d.txt', 'r',encoding='utf8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
print('Found %s word vectors.' % len(embeddings_index))
 
embedding_matrix = np.zeros((len(word_index) + 1, vector_len))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

Found 1193514 word vectors.


In [48]:
from keras.layers import LSTM
early_stopping = EarlyStopping(patience = 2)
embedding_layer = Embedding(len(word_index) + 1,
                            vector_len,
                            weights=[embedding_matrix],
                            input_length=input_len,
                            #batch_size=3,
                            trainable=False)

model_rnn_100 = Sequential()
model_rnn_100.add(embedding_layer)
model_rnn_100.add(LSTM(100))
model_rnn_100.add(Dense(25,activation='relu'))
model_rnn_100.add(Dense(2, activation='sigmoid'))
model_rnn_100.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model_rnn_100.summary())
model_rnn_100.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=10, batch_size = 256, callbacks = [early_stopping])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 40, 100)           51659500  
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_27 (Dense)             (None, 25)                2525      
_________________________________________________________________
dense_28 (Dense)             (None, 2)                 52        
Total params: 51,742,477
Trainable params: 82,977
Non-trainable params: 51,659,500
_________________________________________________________________
None
Train on 1277825 samples, validate on 319456 samples
Epoch 1/10
1277825/1277825 [==============================] - 939s - loss: 0.4319 - acc: 0.7981 - val_loss: 0.4038 - val_acc: 0.8155
Epoch 2/10
1277825/1277825 [==============================] - 937s

#### 200 node GLoVe layer

In [50]:
vector_len = 200
embeddings_index = {}
with open('glove.twitter.27B.200d.txt', 'r',encoding='utf8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
print('Found %s word vectors.' % len(embeddings_index))
 
embedding_matrix = np.zeros((len(word_index) + 1, vector_len))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

Found 1193514 word vectors.


In [51]:
from keras.layers import LSTM
early_stopping = EarlyStopping(patience = 2)
embedding_layer = Embedding(len(word_index) + 1,
                            vector_len,
                            weights=[embedding_matrix],
                            input_length=input_len,
                            #batch_size=3,
                            trainable=False)

model_rnn_200 = Sequential()
model_rnn_200.add(embedding_layer)
model_rnn_200.add(LSTM(100))
model_rnn_200.add(Dense(25,activation='relu'))
model_rnn_200.add(Dense(2, activation='sigmoid'))
model_rnn_200.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model_rnn.summary())
model_rnn_200.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=10, batch_size = 256, callbacks = [early_stopping])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 40, 50)            25829750  
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               60400     
_________________________________________________________________
dense_25 (Dense)             (None, 25)                2525      
_________________________________________________________________
dense_26 (Dense)             (None, 2)                 52        
Total params: 25,892,727
Trainable params: 62,977
Non-trainable params: 25,829,750
_________________________________________________________________
None
Train on 1277825 samples, validate on 319456 samples
Epoch 1/10
1277825/1277825 [==============================] - 1157s - loss: 0.4182 - acc: 0.8068 - val_loss: 0.3939 - val_acc: 0.8213
Epoch 2/10
1277825/1277825 [==============================] - 115

### Convolutional Neural Networks

#### 200 node GloVe

In [54]:
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model

early_stopping = EarlyStopping(patience = 2)
embedding_layer = Embedding(len(word_index) + 1,
                            vector_len,
                            weights=[embedding_matrix],
                            input_length=input_len,
                            #batch_size=3,
                            trainable=False)
model_cnn_200 = Sequential()
model_cnn_200.add(embedding_layer)
model_cnn_200.add(Conv1D(128,5,activation = 'relu'))
model_cnn_200.add(MaxPooling1D(5))
model_cnn_200.add(Flatten())
model_cnn_200.add(Dense(25, activation = 'relu'))
model_cnn_200.add(Dense(2, activation = 'sigmoid'))
model_cnn_200.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model_rnn.summary())
model_cnn_200.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=10, callbacks = [early_stopping])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 40, 50)            25829750  
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               60400     
_________________________________________________________________
dense_25 (Dense)             (None, 25)                2525      
_________________________________________________________________
dense_26 (Dense)             (None, 2)                 52        
Total params: 25,892,727
Trainable params: 62,977
Non-trainable params: 25,829,750
_________________________________________________________________
None
Train on 1277825 samples, validate on 319456 samples
Epoch 1/10
1277825/1277825 [==============================] - 779s - loss: 0.4663 - acc: 0.7770 - val_loss: 0.4488 - val_acc: 0.7876
Epoch 2/10
1277825/1277825 [==============================] - 921s

In [ ]:
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
 
sequence_input = Input(shape=(368,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(9)(x) # global max pooling
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(2, activation='softmax')(x)
 
model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])
 
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=50, batch_size=128)
 
